<a href="https://colab.research.google.com/github/NickStrauch13/image-to-text-embedding/blob/tommy/ViT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving artworks.db to artworks (1).db


In [ ]:
import sqlite3
from sklearn.model_selection import train_test_split

# Connect to the SQLite database
db_path = './artworks.db'  # Adjusted path to the database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Fetch all data from the database
cursor.execute('SELECT image_url, description FROM artworks')
data = cursor.fetchall()

# Convert data to a pandas DataFrame for easier manipulation (optional)
import pandas as pd
df = pd.DataFrame(data, columns=['image_url', 'description'])

# Split the data
train_val_data, test_data = train_test_split(df, test_size=0.15, random_state=42)
train_data, val_data = train_test_split(train_val_data, test_size=(0.15 / 0.85), random_state=42)

# Convert DataFrame back to dictionary if you prefer dict over DataFrame
train_data = {'image_url': train_data['image_url'].tolist(), 'description': train_data['description'].tolist()}
val_data = {'image_url': val_data['image_url'].tolist(), 'description': val_data['description'].tolist()}
test_data = {'image_url': test_data['image_url'].tolist(), 'description': test_data['description'].tolist()}

connection.close()


In [ ]:
import torch
from torch.utils.data import Dataset
from PIL import Image
import requests
from transformers import ViTFeatureExtractor, BertTokenizer
from io import BytesIO

class CaptionDataset(Dataset):
    def __init__(self, datasets):
        self.datasets = datasets

    def __len__(self):
        return len(self.datasets['image_url'])

    def __getitem__(self, idx):
        try:
            response = requests.get(self.datasets['image_url'][idx], stream=True)
            image = Image.open(BytesIO(response.content))
        except Exception as e:
            print(f"Error loading image {self.datasets['image_url'][idx]}: {e}")
            # Using a placeholder image if the actual image fails to load
            image = Image.new('RGB', (224, 224), color='gray')

        image_features = image_processor(images=image, return_tensors="pt").pixel_values
        labels = tokenizer(self.datasets["description"][idx], return_tensors="pt",
                           max_length=128, padding="max_length", truncation=True).input_ids.squeeze()

        return {'pixel_values': image_features.squeeze(), 'labels': labels}


In [ ]:
image_processor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [ ]:
from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor, BertTokenizer

# Load or initialize your model here
# model = VisionEncoderDecoderModel.from_pretrained('your-pretrained-model-name')

# Load your tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Set decoder_start_token_id
model.config.decoder_start_token_id = tokenizer.cls_token_id  # or tokenizer.bos_token_id
model.config.pad_token_id = tokenizer.pad_token_id

In [ ]:
from transformers import VisionEncoderDecoderModel, VisionEncoderDecoderConfig, ViTConfig, BertConfig

config_encoder = ViTConfig()
config_decoder = BertConfig()
config = VisionEncoderDecoderConfig.from_encoder_decoder_configs(config_encoder, config_decoder)
model = VisionEncoderDecoderModel(config=config)


In [ ]:
# Assuming train_data, val_data are prepared as shown previously
datasets_train = CaptionDataset(train_data)
dataset_val = CaptionDataset(val_data)

In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import AdamW

# Assuming `model` has been defined and loaded
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)

train_dataloader = DataLoader(datasets_train, batch_size=8, shuffle=True)
val_dataloader = DataLoader(dataset_val, batch_size=8)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from tqdm.auto import tqdm

num_epochs = 3  # Set the number of epochs

for epoch in range(num_epochs):
    model.train()
    total_train_loss = 0

    # Training loop
    for batch in tqdm(train_dataloader, desc=f"Training Epoch {epoch + 1}"):
        optimizer.zero_grad()
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(pixel_values=batch["pixel_values"], labels=batch["labels"])
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_dataloader)
    print(f"Average training loss: {avg_train_loss}")

    # Evaluation loop
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for batch in tqdm(val_dataloader, desc=f"Evaluating Epoch {epoch + 1}"):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(pixel_values=batch["pixel_values"], labels=batch["labels"])
            loss = outputs.loss
            total_val_loss += loss.item()

    avg_val_loss = total_val_loss / len(val_dataloader)
    print(f"Average validation loss: {avg_val_loss}")


Training Epoch 1:   0%|          | 0/3 [00:00<?, ?it/s]

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Error loading image https://www.artic.edu/iiif/2/6f82c731-2be0-7d16-8bf8-a764a082dc3a/full/843,/0/default.jpg: cannot identify image file <_io.BytesIO object at 0x7fe7a41eb880>
Average training loss: 10.072871526082357


Evaluating Epoch 1:   0%|          | 0/1 [00:00<?, ?it/s]

Average validation loss: 9.587894439697266


...

Average validation loss: 9.336922645568848


Training Epoch 3:   0%|          | 0/3 [00:00<?, ?it/s]

Error loading image https://www.artic.edu/iiif/2/6f82c731-2be0-7d16-8bf8-a764a082dc3a/full/843,/0/default.jpg: cannot identify image file <_io.BytesIO object at 0x7fe78a5f2ac0>
Average training loss: 9.190858205159506


Evaluating Epoch 3:   0%|          | 0/1 [00:00<?, ?it/s]

Average validation loss: 9.11560344696045


# BELOW THIS IS EXPERIMENTAL/EXAMPLE FOUND AT (https://sushantjha8.medium.com/lets-train-image-to-text-transformer-846150b632ef). ABOVE IS ACTUAL IMPLEMENTATION WITH OUR DATASET

In [ ]:
!pip install datasets

In [ ]:
from transformers import ViTImageProcessor, BertTokenizer, VisionEncoderDecoderModel
from datasets import load_dataset

In [ ]:
from transformers import BertConfig, ViTConfig, VisionEncoderDecoderConfig, VisionEncoderDecoderModel

config_encoder = ViTConfig()
config_decoder = BertConfig()
config = VisionEncoderDecoderConfig.from_encoder_decoder_configs(config_encoder, config_decoder)
model = VisionEncoderDecoderModel(config=config)
image_processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from datasets import list_datasets, load_dataset
data=load_dataset('conceptual_captions')
train_data=data["train"]
val_data=data["validation"]

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for conceptual_captions contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conceptual_captions
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [ ]:
from PIL import Image
import requests
import torch

class captiondatset(torch.utils.data.Dataset):
    def __init__(self, datasets):
        self.datasets = datasets
    def __getitem__(self, idx):

      image = Image.open(requests.get(self.datasets['image_url'][idx],stream=True).raw)
      image_features = image_processor(image, return_tensors="pt").pixel_values

      labels = tokenizer(self.datasets["caption"][idx],return_tensors="pt",
                                          max_length=46,
                                          pad_to_max_length=True,
                                          return_token_type_ids=True,
                                          truncation=True).input_ids
      return {'pixel_values':image_features.squeeze(0),'labels':labels.squeeze(0)}
datsets_train = captiondatset(train_data)
dataset_val = captiondatset(val_data)

In [ ]:
!pip install accelerate -U

In [ ]:
import accelerate
print(accelerate.__version__)


0.28.0


In [ ]:
!pip install transformers -U


In [ ]:
class captiondatset(torch.utils.data.Dataset):
    def __init__(self, datasets):
        self.datasets = datasets

    def __len__(self):
        return len(self.datasets['image_url'])  # Assuming 'image_url' is a key in your dataset dict

    def __getitem__(self, idx):
        image = Image.open(requests.get(self.datasets['image_url'][idx], stream=True).raw)
        image_features = image_processor(image, return_tensors="pt").pixel_values

        labels = tokenizer(self.datasets["caption"][idx], return_tensors="pt",
                           max_length=46,
                           padding="max_length",
                           return_token_type_ids=True,
                           truncation=True).input_ids
        return {'pixel_values': image_features.squeeze(0), 'labels': labels.squeeze(0)}


In [ ]:
import accelerate
from transformers import Trainer, TrainingArguments
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model=model.to(device)
model.train()
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=30,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)



trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=datsets_train,         # training dataset
    eval_dataset=dataset_val            # evaluation dataset
)
trainer.train()

ValueError: The train_dataset does not implement __len__, max_steps has to be specified. The number of steps needs to be known in advance for the learning rate scheduler.